In [ ]:
%pip -q install opencv-python-headless gradio matplotlib numpy

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))

In [2]:
import os, io, json, glob, math
import numpy as np
import cv2
import gradio as gr
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# A1 imports
try:
    from calibration.calib_io import load_calibration_json, save_calibration_json
except Exception:
    # fallback tiny loaders to avoid breaking if A1 signatures differ
    def load_calibration_json(path:str):
        with open(path, "r") as f:
            d = json.load(f)
        K = np.array(d["K"], float)
        dist = np.array(d["dist"], float)
        # Fallback: no image size info, so return None
        return K, dist, None
    def save_calibration_json(path:str, K, dist):
        with open(path, "w") as f:
            json.dump({"K": np.asarray(K, float).tolist(),
                       "dist": np.asarray(dist, float).reshape(-1).tolist()}, f, indent=2)

# A2 imports 
from planar_pose.homography import dlt_homography_ransac
from planar_pose.pose_from_homography import pose_from_homography
from planar_pose.opencv_pose import pose_solvepnp, pose_from_h_decompose
from planar_pose.model_points import load_model_points, generate_checkerboard
from planar_pose.viz import overlay_points_and_axes, plot_3d_poses
from planar_pose.compare import rotation_angle_diff, translation_dir_angle, reprojection_error
from planar_pose.clicks import ClickStore

# Repo images listing
def list_repo_images():
    return sorted(glob.glob("data/images/*.*"))

# Draw small numeric labels on a preview image (for click status)
def draw_indexed_points(bgr, pts, color=(0,255,255)):
    out = bgr.copy()
    for i,(u,v) in enumerate(pts):
        cv2.circle(out, (int(u),int(v)), 5, color, -1)
        cv2.putText(out, str(i), (int(u)+6,int(v)-6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
    return out

# Pretty-print matrices
def fmt_rt(R, t):
    R = np.asarray(R, float)
    t = np.asarray(t, float).reshape(3,1)
    return (f"R =\n{np.array2string(R, precision=4, suppress_small=True)}\n\n"
            f"t =\n{np.array2string(t.reshape(-1), precision=4, suppress_small=True)}")

# Small helper: convert Matplotlib fig to bytes
def fig_to_png_bytes(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight", dpi=120)
    plt.close(fig)
    buf.seek(0)
    return buf.read()


c:\Users\arian\.conda\envs\calibration\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def find_corners_on_images(image_files, inner_cols, inner_rows):
    pattern_size = (inner_cols, inner_rows)  # OpenCV INNER corners
    objp = np.zeros((inner_rows*inner_cols,3), np.float32)
    objp[:,:2] = np.mgrid[0:inner_cols, 0:inner_rows].T.reshape(-1,2)

    objpoints, imgpoints = [], []
    out_previews = []
    used_paths = []

    for p in image_files:
        img = cv2.imread(p, cv2.IMREAD_COLOR)
        if img is None: 
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        if not ret:
            out_previews.append(img)
            continue
        # refine
        corners = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        objpoints.append(objp.copy())
        imgpoints.append(corners)
        prev = img.copy()
        cv2.drawChessboardCorners(prev, pattern_size, corners, True)
        out_previews.append(prev)
        used_paths.append(p)

    return objpoints, imgpoints, out_previews, used_paths

def calibrate_chessboard(image_files, inner_cols, inner_rows, square_size_m, preview_size=640):
    objpoints, imgpoints, previews, used_paths = find_corners_on_images(image_files, inner_cols, inner_rows)
    if len(objpoints) < 3:
        return None, None, None, f"Found chessboard on only {len(objpoints)} images. Need >=3."

    # scale objpoints to meters
    for i in range(len(objpoints)):
        objpoints[i][:,:2] *= float(square_size_m)

    img0 = cv2.imread(used_paths[0], cv2.IMREAD_COLOR)
    h, w = img0.shape[:2]
    ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (w,h), None, None)

    # per-image reprojection errors
    per_img_err = []
    for i in range(len(objpoints)):
        imgpts, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, dist)
        e = cv2.norm(imgpoints[i], imgpts, cv2.NORM_L2) / len(imgpts)
        per_img_err.append(float(e))
    rms = float(ret)
    summary = {"rms": rms, "mean_per_image_rmse": float(np.mean(per_img_err)), "num_images": len(objpoints)}

    # resize previews
    out = []
    for im in previews:
        scale = preview_size / max(im.shape[:2])
        im = cv2.resize(im, (int(im.shape[1]*scale), int(im.shape[0]*scale)))
        out.append(im)

    return K, dist, out, json.dumps(summary, indent=2)

def undistort_preview(img_bgr, K, dist, alpha=0.85):
    h, w = img_bgr.shape[:2]
    newK, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w,h), alpha, (w,h), centerPrincipalPoint=True)
    und = cv2.undistort(img_bgr, K, dist, None, newK)
    return und, newK


In [4]:
# Global (per-session) state
click_store = ClickStore()
R_h_t = None   # (R_h, t_h)
R_o_t = None   # (R_o, t_o)
newK_cache = None
current_canvas_rgb = None

def _bytes_from_bgr(bgr):
    ok, buf = cv2.imencode(".png", bgr)
    return buf.tobytes() if ok else None

def list_repo_images():
    # Use absolute path for image listing, compatible with notebook (no __file__)
    img_dir = os.path.abspath(os.path.join(os.getcwd(), "data", "images"))
    return sorted(glob.glob(os.path.join(img_dir, "*.*")))

def fig_to_bgr_array(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight", dpi=120)
    plt.close(fig)
    buf.seek(0)
    arr = np.frombuffer(buf.getvalue(), np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    return img

# ---------- TAB 1: CALIBRATION (A1-like) ----------
def tab1_list_repo_images():
    return list_repo_images()

def tab1_run_calibrate(repo_paths, uploaded_images, inner_cols, inner_rows, square_size_m):
    files = []
    files += repo_paths or []
    files += [f.name for f in uploaded_images] if uploaded_images else []
    files = [p for p in files if os.path.exists(p)]
    if not files:
        return None, None, None, "No images provided."

    K, dist, previews, summary = calibrate_chessboard(files, inner_cols, inner_rows, square_size_m)
    if K is None:
        return None, None, None, summary

    # pass previews as numpy arrays (OpenCV images) for Gradio Gallery
    gallery = [im for im in previews]
    return json.dumps(K.tolist(), indent=2), json.dumps(dist.reshape(-1).tolist(), indent=2), gallery, summary

def tab1_undistort_single(image_file, K_json, dist_json, alpha):
    if image_file is None:
        return None, None
    K = np.array(json.loads(K_json), float)
    dist = np.array(json.loads(dist_json), float).reshape(-1,1)
    # Handle Gradio file input: file-like or string path
    if hasattr(image_file, "read"):
        data = image_file.read()
    else:
        data = open(image_file, "rb").read()
    img = cv2.imdecode(np.frombuffer(data, np.uint8), cv2.IMREAD_COLOR)
    und, newK = undistort_preview(img, K, dist, alpha=float(alpha))
    # Return numpy array for Gradio Image
    return und, json.dumps(newK.tolist(), indent=2)

def tab1_save_intrinsics(save_path, K_json, dist_json):
    try:
        K = np.array(json.loads(K_json), float)
        dist = np.array(json.loads(dist_json), float).reshape(-1,1)
        import inspect
        # Ensure parent directory exists
        parent_dir = os.path.dirname(save_path)
        if parent_dir and not os.path.exists(parent_dir):
            os.makedirs(parent_dir, exist_ok=True)
        num_args = len(inspect.signature(save_calibration_json).parameters)
        if num_args == 3:
            save_calibration_json(save_path, K, dist)
        else:
            # fallback expects (path, dict) but must open file for writing
            with open(save_path, "w") as f:
                json.dump({"K": K.tolist(), "dist": dist.reshape(-1).tolist()}, f, indent=2)
        return f"Saved intrinsics to: {save_path}"
    except Exception as e:
        return f"Failed to save: {e}"

# ---------- TAB 2: MODEL PLANE ----------
def tab2_generate_checker(cols, rows, square_size_m):
    XY = generate_checkerboard(int(cols), int(rows), float(square_size_m))
    # quick preview plot with numbers
    fig = plt.figure(figsize=(3.5,3.5))
    plt.scatter(XY[:,0], XY[:,1], s=20)
    for i,(x,y) in enumerate(XY):
        plt.text(x, y, str(i), fontsize=8)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.title("Model points (Z=0)")
    plt.xlabel("X (m)"); plt.ylabel("Y (m)")
    plt.grid(True, alpha=0.3)
    img_bgr = fig_to_bgr_array(fig)
    return img_bgr, json.dumps(XY.tolist(), indent=2)

def tab2_load_model(file):
    XY = load_model_points(file.name)
    # same preview
    fig = plt.figure(figsize=(3.5,3.5))
    plt.scatter(XY[:,0], XY[:,1], s=20)
    for i,(x,y) in enumerate(XY):
        plt.text(x, y, str(i), fontsize=8)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.title("Loaded model points")
    plt.xlabel("X (m)"); plt.ylabel("Y (m)")
    plt.grid(True, alpha=0.3)
    img_bgr = fig_to_bgr_array(fig)
    return img_bgr, json.dumps(XY.tolist(), indent=2)

def tab2_save_model(save_path, XY_json):
    try:
        XY = np.array(json.loads(XY_json), float)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        np.savetxt(save_path, XY, delimiter=",", fmt="%.8f")
        return f"Saved model points CSV: {save_path}"
    except Exception as e:
        return f"Failed to save model: {e}"

# ---------- TAB 3: POSE ----------
def tab3_reset_clicks():
    click_store.clear()
    return "Cleared clicks."

def tab3_undo_click():
    click_store.undo()
    return f"Points: {list(click_store)}"

# --- Point Picker Handler Replacement ---
def tab3_on_click(evt: gr.SelectData):
    global current_canvas_rgb
    x = y = None

    if evt is not None and hasattr(evt, "index") and evt.index is not None:
        try:
            x, y = evt.index
        except Exception:
            x = y = None
    if (x is None or y is None) and evt is not None:
        if hasattr(evt, "x") and hasattr(evt, "y"):
            x, y = evt.x, evt.y

    if current_canvas_rgb is None:
        return gr.update(), gr.update()
    if x is None or y is None:
        return current_canvas_rgb, current_canvas_rgb

    h, w = current_canvas_rgb.shape[:2]
    u = int(max(0, min(w - 1, x)))
    v = int(max(0, min(h - 1, y)))
    click_store.add(u, v)  # stores (u, v) for downstream pose, as designed :contentReference[oaicite:0]{index=0}

    bgr = cv2.cvtColor(current_canvas_rgb, cv2.COLOR_RGB2BGR)
    vis_bgr = draw_indexed_points(bgr, list(click_store))
    vis_rgb = cv2.cvtColor(vis_bgr, cv2.COLOR_BGR2RGB)
    return vis_rgb, vis_rgb

def tab3_load_to_canvas(image_file, intr_file):
    global current_canvas_rgb
    click_store.clear()
    if image_file is None:
        return None, "Load an image first."
    data = image_file.read() if hasattr(image_file, "read") else open(str(image_file), "rb").read()
    bgr = cv2.imdecode(np.frombuffer(data, np.uint8), cv2.IMREAD_COLOR)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    current_canvas_rgb = rgb.copy()            # <-- store for clicks
    return rgb, "Image loaded. Click points in the same order as the model."

def _prepare_image(image_file, K, dist):
    if hasattr(image_file, "read"):
        data = image_file.read()
    else:
        data = open(str(image_file), "rb").read()
    img = cv2.imdecode(np.frombuffer(data, np.uint8), cv2.IMREAD_COLOR)
    return img, K, dist

def tab3_run_pose(image_file, intrinsics_file, model_file, ransac_thresh, solver_choice):
    global R_h_t, R_o_t
    if image_file is None or intrinsics_file is None or model_file is None:
        return None, None, "Provide image, intrinsics, and model points.", None

    # load K, dist, XY, and clicks
    result = load_calibration_json(intrinsics_file.name)
    if isinstance(result, tuple):
        K, dist, *rest = result
        img_size = rest[0] if rest else None
    else:
        # fallback: assume dict
        K = np.array(result["K"], float)
        dist = np.array(result["dist"], float)
        img_size = None

    XY = load_model_points(model_file.name)
    uv = click_store.as_array(float)
    if len(uv) != len(XY):
        return None, None, f"Need {len(XY)} clicks; you have {len(uv)}.", None

    # prepare image & intrinsics for projection consistency
    img, K_used, dist_used = _prepare_image(image_file, K, dist)
    # compute H with RANSAC
    H, inliers = dlt_homography_ransac(XY, uv, float(ransac_thresh))

    # Homography → Pose
    R_h, t_h = pose_from_homography(H, K_used)
    # OpenCV pose
    if solver_choice == "solvePnP":
        R_o, t_o = pose_solvepnp(K_used, dist_used, XY, uv)
    elif solver_choice == "decomposeH":
        R_o, t_o = pose_from_h_decompose(H, K_used, XY)
    else:
        # run both and choose solvePnP for printing
        R_o, t_o = pose_solvepnp(K_used, dist_used, XY, uv)

    # Overlays
    vis_h = overlay_points_and_axes(img, K_used, dist_used, R_h, t_h, XY)
    vis_o = overlay_points_and_axes(img, K_used, dist_used, R_o, t_o, XY)

    # Errors
    err_h = reprojection_error(K_used, dist_used, R_h, t_h, XY, uv)
    err_o = reprojection_error(K_used, dist_used, R_o, t_o, XY, uv)

    # Save in app state for Tab 4
    R_h_t = (R_h, t_h)
    R_o_t = (R_o, t_o)

    # text
    text = (
        "[Homography → Pose]\n" + fmt_rt(R_h,t_h) +
        f"\nErrors(px): mean={err_h['mean']:.3f}, median={err_h['median']:.3f}, max={err_h['max']:.3f}\n"
        f"Inliers: {int(np.sum(inliers))}/{len(inliers)}\n\n"
        "[OpenCV Pose]\n" + fmt_rt(R_o,t_o) +
        f"\nErrors(px): mean={err_o['mean']:.3f}, median={err_o['median']:.3f}, max={err_o['max']:.3f}"
    )

    # Return numpy arrays for Gradio Image
    return vis_h, vis_o, text, f"Stored 2 poses for comparison."

# ---------- TAB 4: COMPARE & 3D ----------
def tab4_compare_and_viz():
    if R_h_t is None and R_o_t is None:
        return None, "No poses computed yet."
    items = []
    if R_h_t and R_o_t:
        R_h, t_h = R_h_t
        R_o, t_o = R_o_t
        dR = rotation_angle_diff(R_h, R_o)
        dt = translation_dir_angle(t_h, t_o)
        txt = f"Rotation angle difference: {dR:.3f}°\nTranslation direction angle: {dt:.3f}°"
        fig = plot_3d_poses([R_h_t, R_o_t], plane_extent=0.2)
        img_bgr = fig_to_bgr_array(fig)
        return img_bgr, txt
    else:
        fig = plot_3d_poses([R_h_t or R_o_t], plane_extent=0.2)
        who = "Homography pose" if R_h_t else "OpenCV pose"
        img_bgr = fig_to_bgr_array(fig)
        return img_bgr, f"Only {who} available."

# ---------- APP ----------
with gr.Blocks() as demo:
    gr.Markdown("# Assignment 2: Pose from a Planar Object (Tabs)\n*No changes to Assignment 1 files. This app reuses A1 artifacts and adds A2 tabs.*")

    with gr.Tab("1) Calibration (A1-like)"):
        with gr.Row():
            repo_imgs = gr.CheckboxGroup(choices=tab1_list_repo_images(), label="Repo images (data/images/)")
            uploads = gr.Files(label="Upload images")
        with gr.Row():
            inner_cols = gr.Number(value=9, precision=0, label="Inner cols")
            inner_rows = gr.Number(value=6, precision=0, label="Inner rows")
            square_m  = gr.Number(value=0.022, label="Square size (m)")
        run_cal = gr.Button("Run calibration")
        K_txt = gr.Textbox(label="K (JSON)", lines=6)
        dist_txt = gr.Textbox(label="distCoeffs (JSON)", lines=3)
        gallery = gr.Gallery(label="Corners preview")
        summary = gr.Textbox(label="Summary (RMS etc.)", lines=6)

        run_cal.click(tab1_run_calibrate, 
                      inputs=[repo_imgs, uploads, inner_cols, inner_rows, square_m],
                      outputs=[K_txt, dist_txt, gallery, summary])

        gr.Markdown("### Undistort preview")
        und_src = gr.File(label="Image to undistort")
        alpha = gr.Slider(0.0, 1.0, value=0.85, step=0.05, label="Alpha (balance)")
        und_out = gr.Image(label="Undistorted")
        newK_txt = gr.Textbox(label="newK (JSON)", lines=6)
        und_btn = gr.Button("Undistort this image")
        und_btn.click(tab1_undistort_single, inputs=[und_src, K_txt, dist_txt, alpha], outputs=[und_out, newK_txt])

        gr.Markdown("### Save intrinsics JSON")
        save_path = gr.Textbox(value="sample_data/intrinsics_sample.json", label="Save path")
        save_btn = gr.Button("Save intrinsics")
        save_status = gr.Textbox(label="Save status")
        save_btn.click(tab1_save_intrinsics, inputs=[save_path, K_txt, dist_txt], outputs=[save_status])

    with gr.Tab("2) Model Plane"):
        with gr.Row():
            cols = gr.Number(value=9, precision=0, label="Inner cols")
            rows = gr.Number(value=6, precision=0, label="Inner rows")
            size_m = gr.Number(value=0.022, label="Square size (m)")
        gen_btn = gr.Button("Generate checkerboard model")
        model_preview = gr.Image(label="Model preview")
        model_json = gr.Textbox(label="Model XY (JSON)", lines=8)
        gen_btn.click(tab2_generate_checker, inputs=[cols, rows, size_m], outputs=[model_preview, model_json])

        gr.Markdown("**Or upload CSV/JSON**")
        model_file = gr.File(label="CSV (X,Y) or JSON ([[X,Y],...])")
        load_btn = gr.Button("Load model")
        load_btn.click(tab2_load_model, inputs=[model_file], outputs=[model_preview, model_json])

        save_model_path = gr.Textbox(value="sample_data/model_points.csv", label="Save model CSV")
        save_model_btn = gr.Button("Save model points")
        save_model_status = gr.Textbox(label="Save status")
        save_model_btn.click(tab2_save_model, inputs=[save_model_path, model_json], outputs=[save_model_status])

    with gr.Tab("3) Pose on Image"):
        img_file = gr.File(label="Planar scene image")

        intr_file = gr.File(label="Intrinsics JSON (K, dist)")

        mdl_file = gr.File(label="Model points (CSV/JSON)")
        with gr.Row():
            ransac = gr.Slider(0.5, 10.0, value=3.0, step=0.5, label="RANSAC threshold (px)")
            solver = gr.Radio(["solvePnP", "decomposeH", "both"], value="both", label="OpenCV solver")

        status = gr.Textbox(label="Click status / messages")
        # Make canvas explicitly numpy type
        img_canvas = gr.Image(type="numpy", label="Click here in order", interactive=True)

        with gr.Row():
            clear_btn = gr.Button("Clear clicks")
            undo_btn  = gr.Button("Undo last click")

        img_file.change(tab3_load_to_canvas, inputs=[img_file, intr_file], outputs=[img_canvas, status])
        def tab3_on_click_and_json(evt: gr.SelectData):
            vis_rgb, _ = tab3_on_click(evt)
            pts_json = json.dumps(list(click_store))
            return vis_rgb, vis_rgb, pts_json
        pts_text = gr.Textbox(label="Points (JSON)", lines=3)
        img_canvas.select(tab3_on_click_and_json, outputs=[img_canvas, img_canvas, pts_text])

        def tab3_reset_clicks_and_json():
            click_store.clear()
            return "Cleared clicks.", json.dumps(list(click_store))
        clear_btn.click(tab3_reset_clicks_and_json, outputs=[status, pts_text])

        def tab3_undo_click_and_json():
            click_store.undo()
            return f"Points: {list(click_store)}", json.dumps(list(click_store))
        undo_btn.click(tab3_undo_click_and_json, outputs=[status, pts_text])

        def tab3_load_to_canvas_and_json(image_file, intr_file):
            global current_canvas_rgb
            click_store.clear()
            if image_file is None:
                return None, "Load an image first.", json.dumps(list(click_store))
            data = image_file.read() if hasattr(image_file, "read") else open(str(image_file), "rb").read()
            bgr = cv2.imdecode(np.frombuffer(data, np.uint8), cv2.IMREAD_COLOR)
            rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            current_canvas_rgb = rgb.copy()
            return rgb, "Image loaded. Click points in the same order as the model.", json.dumps(list(click_store))
        img_file.change(tab3_load_to_canvas_and_json, inputs=[img_file, intr_file], outputs=[img_canvas, status, pts_text])

        run_pose_btn = gr.Button("Run pose (compute overlays)")
        vis_h = gr.Image(label="Homography overlay")
        vis_o = gr.Image(label="OpenCV overlay")
        pose_txt = gr.Textbox(label="Poses and errors", lines=14)
        stored_msg = gr.Textbox(label="Stored state for Tab 4")

        run_pose_btn.click(tab3_run_pose, inputs=[img_file, intr_file, mdl_file, ransac, solver], outputs=[vis_h, vis_o, pose_txt, stored_msg])

    with gr.Tab("4) Compare & 3D Viz"):
        cmp_btn = gr.Button("Show comparison")
        cmp_fig = gr.Image(label="3D viz")
        cmp_txt = gr.Textbox(label="Numeric comparison", lines=4)
        cmp_btn.click(lambda: tab4_compare_and_viz(), outputs=[cmp_fig, cmp_txt])

    demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://925e594a120e4f8c2e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
* Running on public URL: https://925e594a120e4f8c2e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://925e594a120e4f8c2e.gradio.live


# 

### Comparison notes (Assignment 2)

- **Accuracy:** OpenCV (solvePnP) is tighter - mean reproj **1.526 px** (median 1.264, max 3.909) vs. Homography→Pose **5.269 px** (median 4.789, max 11.551).
- **Pose agreement:** Rotation diff **2.983°**; translation **direction** diff **0.034°** → orientations and pointing direction match well.
- **Why the gap:** We estimated **H** on the raw (distorted) image; homography assumes pinhole mapping, so residual distortion becomes model error. `solvePnP` uses `distCoeffs`, explaining distortion and staying tighter.
- **RANSAC helped:** **47/54** inliers at 3 px — rejects a few bad/edge clicks; both pipelines become stable.
- **More points ≫ 4 corners:** Using **54** well-spread points improves conditioning and averages click noise; with only four, H is noticeably noisier.
- **Scale/translation:** With model points in meters, both methods yield compatible translation magnitudes; homography shows a bit more scale jitter across runs.
- **Numerical hygiene:** SVD re-orthonormalization and enforcing **det(R)=+1** are important for Homography→Pose; skipping them skews \(R\) and worsens overlays.
- **Decomposition ambiguity:** `decomposeHomographyMat` can return multiple poses; `solvePnP` avoids cheirality selection and was the more reliable OpenCV baseline here.
